In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
#import tensorflow_io as tfio

from tensorflow import keras
from scipy import signal
from scipy.io import wavfile

In [2]:
DIREC = 'TRAIN_V2/data_out_2/'

SAMPLING_RATE = 16000

In [3]:
def path_to_audio(path):
    """Reads and decodes an audio file."""
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio

In [4]:
#read the files in train_files
train_files = os.listdir(DIREC + str(18) + '/')
for i in range(len(train_files)):
    sample_rate, samples = wavfile.read(DIREC + str(18) + '/' + train_files[i])
    audio = path_to_audio(DIREC + str(18) + '/' + train_files[i])
    print(audio)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    #plt.pcolormesh(times, frequencies, spectrogram)
    #plt.imshow(spectrogram)
    #plt.ylabel('Frequency [Hz]')
    #plt.xlabel('Time [sec]')
    #plt.show()

tf.Tensor(
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 ...
 [-0.06625366]
 [-0.09240723]
 [-0.04507446]], shape=(16000, 1), dtype=float32)
tf.Tensor(
[[0.08862305]
 [0.13967896]
 [0.14242554]
 ...
 [0.09439087]
 [0.08599854]
 [0.08209229]], shape=(16000, 1), dtype=float32)
tf.Tensor(
[[ 0.0000000e+00]
 [-3.0517578e-05]
 [-3.0517578e-05]
 ...
 [-3.1677246e-02]
 [-2.9846191e-02]
 [-2.9205322e-02]], shape=(16000, 1), dtype=float32)
tf.Tensor(
[[-3.0517578e-05]
 [-3.0517578e-05]
 [-3.0517578e-05]
 ...
 [-6.9580078e-03]
 [ 5.2612305e-02]
 [ 1.2603760e-02]], shape=(16000, 1), dtype=float32)
tf.Tensor(
[[ 0.08331299]
 [ 0.13070679]
 [ 0.12637329]
 ...
 [-0.04928589]
 [-0.04455566]
 [-0.03970337]], shape=(16000, 1), dtype=float32)
tf.Tensor(
[[ 0.0000000e+00]
 [-3.0517578e-05]
 [ 0.0000000e+00]
 ...
 [-1.3131714e-01]
 [-1.9430542e-01]
 [-1.6000366e-01]], shape=(16000, 1), dtype=float32)
tf.Tensor(
[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 ...
 [-0.2548523 ]
 [-0.35671997]
 [-0.34680176]]